# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on Structured Streaming** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

#### Connection with Spark cluster


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Sockets") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Example with "rate" format

In [ ]:
rate_df = spark \
            .readStream \
            .format("rate") \
            .load()

rate_df.printSchema()

In [ ]:
query = rate_df.filter(rate_df.value % 2 == 1) \
                .writeStream \
                .outputMode("append") \
                .format("console") \
                .start()

In [ ]:
query.stop()

###  Example with "socket" format

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

lines = spark \
            .readStream \
            .format("socket") \
            .option("host", "localhost") \
            .option("port", "9999") \
            .load()

In [ ]:
from pyspark.sql.functions import explode, split

# Simple processing example: word counter
words = lines.select(explode(split(lines.value, " ")).alias("word"))
word_count = words.groupBy("word").count()

query = word_count \
        .writeStream \
        .outputMode("complete") \
        .format("console") \
        .start()

query.awaitTermination()

In [ ]:
sc.stop()